# Cesium

In [1]:
#!pip install cesium
# !pip install pandas==1.0.5

In [1]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import scipy.stats as ss

---

In [3]:
# create some dummy data
data_dir = Path("../data")

df_acc = pd.read_parquet(
    data_dir.joinpath("empatica/acc.parquet"), engine="fastparquet"
).set_index("timestamp")

fs = 1000  # the sample frequency
duration_s = 1 * 60 * 60  # 1 hour of data
size = int(duration_s * fs)

df_emg = pd.DataFrame(
    index=pd.date_range(
        start=datetime.now(), periods=size, freq=pd.Timedelta(seconds=1 / fs)
    ),
    data=np.array(
        [
            np.repeat(df_acc.values[:, idx % 3] / 64, np.ceil(size / len(df_acc)))[
                :size
            ]
            for idx in range(5)
        ]
    ).astype(np.float32).transpose(),
    columns=["emg", "eog", "lso", "rio", "m1-a1"],
)
print("memory usage: ", round(sum(df_emg.memory_usage(deep=True) / (2**20)), 2), "MB")
df_emg.tail(3)

memory usage:  96.13 MB


,emg,eog,lso,rio,m1-a1
2021-06-24 16:09:41.808056,0.34375,-0.90625,0.171875,0.34375,-0.90625
2021-06-24 16:09:41.809056,0.34375,-0.90625,0.171875,0.34375,-0.90625
2021-06-24 16:09:41.810056,0.34375,-0.90625,0.171875,0.34375,-0.90625


In [4]:
from cesium import featurize, util, data_management, time_series, datasets

In [5]:
eeg = datasets.fetch_andrzejak()

# Group together classes (Z, O), (N, F), (S) as normal, interictal, ictal
eeg["classes"] = eeg["classes"].astype("U16") #  allocate memory for longer class names
eeg["classes"][np.logical_or(eeg["classes"]=="Z", eeg["classes"]=="O")] = "Normal"
eeg["classes"][np.logical_or(eeg["classes"]=="N", eeg["classes"]=="F")] = "Interictal"
eeg["classes"][eeg["classes"]=="S"] = "Ictal"

Loaded data from cached archive.


In [6]:
pd.DataFrame(eeg)

,times,measurements,classes,archive,header
0,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[40.0, 48.0, 35.0, 5.0, -40.0, -54.0, -32.0, 6...",Normal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...
1,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[-56.0, -50.0, -64.0, -91.0, -135.0, -140.0, -...",Normal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...
2,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[-37.0, -22.0, -17.0, -24.0, -31.0, -20.0, -5....",Normal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...
3,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[-31.0, -43.0, -39.0, -39.0, -9.0, -5.0, 18.0,...",Normal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...
4,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[14.0, 26.0, 32.0, 25.0, 16.0, 8.0, 8.0, 12.0,...",Normal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...
...,...,...,...,...,...
495,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[343.0, 311.0, 284.0, 274.0, 260.0, 237.0, 165...",Ictal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...
496,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[84.0, 75.0, 21.0, -68.0, -138.0, -184.0, -197...",Ictal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...
497,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[-310.0, 93.0, 494.0, 789.0, 798.0, 552.0, 202...",Ictal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...
498,"[0.0, 0.00576171875, 0.0115234375, 0.017285156...","[340.0, 353.0, 400.0, 470.0, 538.0, 590.0, 611...",Ictal,/users/jonvdrdo/.local/datasets/andrzejak/andr...,/users/jonvdrdo/.local/datasets/andrzejak/andr...


## univariate seres where we mingle :clown:

In [7]:
from cesium import featurize

features_to_use = [
    "amplitude",
    "percent_beyond_1_std",
    "maximum",
    "max_slope",
    "median",
    "median_absolute_deviation",
    "percent_close_to_median",
    "minimum",
    "skew",
    "std",
    "weighted_average",
]

# copy the eeg sample dict
eeg_mingled = eeg.copy()

# create a new view where each 
for idx in np.random.choice(len(eeg['measurements']), size=300, replace=False):
    times = eeg_mingled["times"][idx]
    measurements = eeg_mingled["measurements"][idx]
    new_length = np.random.choice(len(times), size=1)[0]
    eeg_mingled["times"][idx] = times[:new_length]
    eeg_mingled["measurements"][idx] = measurements[:new_length]

In [8]:
for t in eeg_mingled['times'][:25]:
    print(t.shape)

(3318,)
(4097,)
(1726,)
(4097,)
(1166,)
(3193,)
(1026,)
(2455,)
(3113,)
(3816,)
(4097,)
(4097,)
(860,)
(4097,)
(4097,)
(1847,)
(4097,)
(4097,)
(4097,)
(4097,)
(4097,)
(2701,)
(332,)
(2074,)
(4097,)


In [10]:
#%%memit
fset_cesium = featurize.featurize_time_series(
    times=eeg_mingled["times"],
    values=eeg_mingled["measurements"],
    errors=None,
    features_to_use=features_to_use,
)

In [17]:
fset_cesium.

feature,amplitude,percent_beyond_1_std,maximum,max_slope,median,median_absolute_deviation,percent_close_to_median,minimum,skew,std,weighted_average
channel,0,0,0,0,0,0,0,0,0,0,0
0,129.5,0.312281,124.0,10066.440678,-4.0,28.0,0.459649,-135.0,-0.023149,41.973427,-4.462456
1,211.5,0.290212,169.0,20653.559322,-51.0,32.0,0.640469,-254.0,-0.092715,48.812668,-52.444716
2,165.0,0.302660,184.0,13537.627119,13.0,31.0,0.515987,-146.0,-0.004100,47.144789,12.705150
3,171.5,0.301253,162.0,17008.813559,-4.0,31.0,0.538650,-181.0,0.117909,47.514800,-3.769325
4,170.0,0.305101,152.0,13016.949153,-18.0,29.0,0.566268,-188.0,0.142753,44.910958,-17.999268
...,...,...,...,...,...,...,...,...,...,...,...
495,876.5,0.368318,727.0,94242.711864,83.0,246.0,0.364413,-1026.0,-0.472757,332.455418,12.870393
496,433.0,0.360459,467.0,27595.932203,11.0,112.0,0.397194,-399.0,0.047599,159.980849,7.912755
497,1359.0,0.267752,1435.0,135202.711864,92.0,194.0,0.621498,-1283.0,-0.400211,401.173451,32.445603


## Custom one to many features


In [138]:
def mean_std_signal(t, m, e):
    return [np.mean(m), np.std(m)]

def skew_signal(t, m, e):
    return ss.skew(m)

In [140]:
guo_features = {
    "mean_std": mean_std_signal,
    "skew": skew_signal
}

fset_guo = featurize.featurize_time_series(times=eeg["times"], values=eeg["measurements"],
                                           errors=None,
                                           features_to_use=list(guo_features.keys()),
                                          # meta_features== features which are added to the output
                                           custom_functions=guo_features)

ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.

---

## Multivariate series where we mingle

In [21]:
!pip install PyWavelets

     |████████████████████████████████| 4.4 MB 4.5 MB/s eta 0:00:01
You should consider upgrading via the '/users/jonvdrdo/jonas/projects/context_aware_health_monitoring/.caw_venv37/bin/python3.7 -m pip install --upgrade pip' command.


In [123]:
import pywt

# create a view with 3 channels
n_channels = 3
eeg["dwts"] = [pywt.wavedec(m, pywt.Wavelet("db1"), level=n_channels-1)
               for m in eeg["measurements"]]

# copy the eeg sample dict
eeg_mingled = eeg.copy()

# downsize the dimensions randomly
for idx in np.random.choice(len(eeg["dwts"]), size=250, replace=False):
    for i in range(3):
        new_length = np.random.choice(len(eeg_mingled["dwts"][idx][i]), size=1)[0] + 20
        eeg_mingled["dwts"][idx][i] = eeg_mingled["dwts"][idx][i][:new_length]

In [128]:
# validation -> print some of these dimensions
for dwts in eeg_mingled['dwts'][:10]:
    print("shapes: ", ", ".join([str(len(dwt)) for dwt in dwts]))

shapes:  208, 257, 701
shapes:  837, 607, 548
shapes:  1025, 1025, 2049
shapes:  819, 819, 1637
shapes:  825, 99, 1710
shapes:  79, 94, 206
shapes:  84, 27, 406
shapes:  1025, 1025, 2049
shapes:  320, 913, 520
shapes:  528, 528, 1056


In [125]:
# try to calculate the features
fset_dwt = featurize.featurize_time_series(
    times=None,
    values=eeg_mingled["dwts"],
    errors=None,
    features_to_use=features_to_use,
)
fset_dwt.head()

feature amplitude                    percent_beyond_1_std                      \
channel         0       1          2                    0         1         2   
0          225.00   83.00  38.537320             0.336538  0.334630  0.319544   
1          328.25  111.50  47.729708             0.303465  0.303130  0.295620   
2          296.50  103.25  44.194174             0.313171  0.311220  0.317716   
3          293.25  125.00  62.578950             0.306471  0.305250  0.299939   
4          293.00   74.00  47.022601             0.311515  0.323232  0.296491   

feature maximum                   max_slope  ...    minimum      skew  \
channel       0      1          2         0  ...          2         0   
0         219.0   81.5  40.305087   61272.0  ... -36.769553 -0.137990   
1         240.5  113.5  51.618795  332728.0  ... -43.840620 -0.014215   
2         322.5  102.5  43.133514  337408.0  ... -45.254834  0.016032   
3         276.5  135.5  69.296465  308795.0  ... -55.861436  0.036011   
4         245.0   64.5  53.033009  321772.0  ... -41.012193  0.031942   

feature                            std                       weighted_average  \
channel         1         2          0          1          2                0   
0       -0.016016  0.138158  86.613517  27.834958  10.529175        -7.262019   
1       -0.031457 -0.082448  87.900778  33.439561  13.052576      -105.698327   
2       -0.061304 -0.136422  88.635399  28.088980  11.246134        25.511707   
3        0.029335  0.000567  84.814675  35.939063  16.426497        -7.583028   
4       -0.486624  0.042338  80.861344  25.705602  11.447219       -39.236970   

feature                      
channel         1         2  
0        0.165370  0.093810  
1       -0.035420 -0.040001  
2       -0.101951  0.007937  
3       -0.006716  0.118787  
4        0.585859 -0.042592  

[5 rows x 33 columns]